In [1]:
# Parameters
test = True


In [2]:
# # Param
# test = "True"

In [3]:
is_test = True
if test == "False" or test == False:
    is_test = False

In [4]:
import os
print(os.getcwd())

/Users/dauduchieu/Documents/iSE2025/CBM


In [5]:
is_test = True

In [6]:
# Param
dataset = 'data'

In [7]:
from utils.data_loader import DataLoader

In [8]:
data_loader = DataLoader(dataset)

In [9]:
train_df = data_loader.get_data_train()

In [10]:
data_desc = data_loader.get_data_desc()
label_column = data_desc['label_column']
text_column = data_desc['text_column']

In [11]:
keyword_concepts = data_loader.get_keyword_concepts()
abstract_concepts = data_loader.get_abstract_concepts()

In [12]:
if is_test:
    train_df = train_df.groupby(label_column).sample(50)

In [13]:
from abc import ABC, abstractmethod
class LLMCaller(ABC):
    @abstractmethod
    def structed_output(self, prompt:str, output_struct):
        pass

from time import time, sleep
from typing import List, Callable, TypeVar

T = TypeVar('T')

class GeminiRateLimiter:
    def __init__(self, requests_per_minute: int = 15):
        self.rpm = requests_per_minute
        self.times: List[float] = []

    def wait(self):
        now = time()
        self.times = [t for t in self.times if now - t <= 60]
        if len(self.times) >= self.rpm:
            sleep(60 - (now - self.times[0]))
            now = time()
            self.times = [t for t in self.times if now - t <= 60]

    def record(self):
        self.times.append(time())

    def execute(self, f: Callable[..., T], *args, **kwargs) -> T:
        self.wait()
        try:
            r = f(*args, **kwargs)
            self.record()
            return r
        except:
            self.record()
            raise

    def __call__(self, f: Callable[..., T]) -> Callable[..., T]:
        def wrapper(*args, **kwargs):
            return self.execute(f, *args, **kwargs)
        return wrapper

    def __enter__(self):
        self.wait()
        return self

    def __exit__(self, *args):
        self.record()

from google import genai

class GeminiAPICaller(LLMCaller):
    def __init__(self, api_key:str, api_model:str, api_rpm:int):
        self.client = genai.Client(api_key=api_key)
        self.api_model = api_model
        self.rate_limiter = GeminiRateLimiter(requests_per_minute=api_rpm)

    def structed_output(self, prompt:str, output_struct):
        with self.rate_limiter:
            response = self.client.models.generate_content(
                model=self.api_model,
                contents=prompt,
                config={
                    'response_mime_type': 'application/json',
                    'response_schema': output_struct,
                },
            )

        res = response.parsed
        return res
    
llm_api_config = data_loader.get_llm_config()

llm_caller = GeminiAPICaller(
    api_key=llm_api_config['api_key'],
    api_model=llm_api_config['model'],
    api_rpm=llm_api_config['rate_per_minute']
)

In [14]:
import pandas as pd
import spacy
from tqdm import tqdm

In [15]:
nlp = spacy.load("en_core_web_sm")

In [16]:
tqdm.pandas(desc="Lemmatizing")
def lemmatize_text(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])
train_df['text_lemma'] = train_df[text_column].progress_apply(lemmatize_text)

Lemmatizing:   0%|                                                                                                                                                    | 0/250 [00:00<?, ?it/s]

Lemmatizing:   2%|██▊                                                                                                                                         | 5/250 [00:00<00:06, 36.38it/s]

Lemmatizing:   4%|█████                                                                                                                                       | 9/250 [00:00<00:08, 29.09it/s]

Lemmatizing:   5%|██████▋                                                                                                                                    | 12/250 [00:00<00:08, 27.54it/s]

Lemmatizing:   6%|████████▉                                                                                                                                  | 16/250 [00:00<00:08, 26.96it/s]

...

Lemmatizing:  95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊       | 237/250 [00:07<00:00, 39.82it/s]

Lemmatizing:  97%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌    | 242/250 [00:07<00:00, 34.56it/s]

Lemmatizing:  98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊  | 246/250 [00:07<00:00, 32.31it/s]

Lemmatizing: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:07<00:00, 28.27it/s]

Lemmatizing: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:07<00:00, 31.58it/s]

In [17]:
keywords = []
for k in keyword_concepts.keys():
    keywords += keyword_concepts[k]

print(keywords)

['hypertension', 'myocardial', 'infarction', 'ischemia', 'coronary', 'pancreatitis', 'ulcer', 'obstruction', 'biliary', 'bowel', 'deficiency', 'disorder', 'necrosis', 'oncogene', 'bone', 'tumor', 'carcinoma', 'adenocarcinoma', 'malignant', 'metastasis', 'neurologic', 'epilepsy', 'seizure', 'spinal', 'cord']


In [18]:
def llm_get_synonyms(data_topic, keyword, n_syn=10):
    prompt = f"""
You are an expert in the {data_topic} domain.

Please list {n_syn} synonyms or alternative expressions for the term: "{keyword}".

The synonyms should be relevant to the {data_topic} context. If the term has multiple meanings, only return synonyms that are appropriate within this context.

Only return a list of synonyms as an array of strings, no explanation.
"""

    output_struct = {
        "type": "array",
        "items": {
            "type": "string"
        }
    }

    synonyms = llm_caller.structed_output(
        prompt=prompt.strip(),
        output_struct=output_struct
    )

    synonyms = [lemmatize_text(t) for t in synonyms]

    return synonyms

In [19]:
print(llm_get_synonyms(
    data_topic=data_desc['data_topic'],
    keyword='hypertension',
    n_syn=5
))

['high blood pressure', 'elevated blood pressure', 'arterial hypertension', 'essential hypertension', 'secondary hypertension']


In [20]:
kw_synonym_dict = {}
for kw in tqdm(keywords, total=len(keywords), desc="Create synonym dict"):
    llm_syn = llm_get_synonyms(data_topic=data_desc['data_topic'], keyword=kw, n_syn=5)
    llm_syn = [lemmatize_text(s) for s in llm_syn]
    kw_synonym_dict.update({
        f"{kw}": [kw] + llm_syn
    })

print(kw_synonym_dict)

Create synonym dict:   0%|                                                                                                                                             | 0/25 [00:00<?, ?it/s]

Create synonym dict:   4%|█████▎                                                                                                                               | 1/25 [00:01<00:24,  1.04s/it]

Create synonym dict:   8%|██████████▋                                                                                                                          | 2/25 [00:01<00:20,  1.15it/s]

Create synonym dict:  12%|███████████████▉                                                                                                                     | 3/25 [00:02<00:19,  1.15it/s]

Create synonym dict:  16%|█████████████████████▎                                                                                                               | 4/25 [00:03<00:17,  1.22it/s]

...

Create synonym dict:  68%|█████████████████████████████████████████████████████████████████████████████████████████▊                                          | 17/25 [01:08<00:18,  2.29s/it]

Create synonym dict:  72%|███████████████████████████████████████████████████████████████████████████████████████████████                                     | 18/25 [01:09<00:13,  1.93s/it]

Create synonym dict:  76%|████████████████████████████████████████████████████████████████████████████████████████████████████▎                               | 19/25 [01:10<00:09,  1.53s/it]

Create synonym dict:  76%|████████████████████████████████████████████████████████████████████████████████████████████████████▎                               | 19/25 [01:31<00:29,  4.84s/it]

KeyboardInterrupt: 

In [ ]:
texts = train_df['text_lemma']
print(len(texts))
print(keywords)
print(kw_synonym_dict)

In [ ]:
def keyword_presence_matrix_from_df(df, keywords, kw_synonym_dict,
                                    text_col='text_column', lemma_col='text_lemma'):
    results = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Keyword weak labeling"):
        original_text = row[text_col]
        lemmatized_text = row[lemma_col]

        for kw in keywords:
            syns = kw_synonym_dict.get(kw, [])
            all_terms = [kw] + syns
            score = int(any(term in lemmatized_text for term in all_terms))
            results.append((original_text, kw, score))

    return pd.DataFrame(results, columns=["text", "keyword", "score"])


In [ ]:
kw_wl_df = keyword_presence_matrix_from_df(train_df, keywords, kw_synonym_dict, text_column)

In [ ]:
kw_wl_df.sample(5)

In [ ]:
kw_wl_df.value_counts('score')

In [ ]:
print(abstract_concepts)

In [ ]:
def aggregate_full_concept_matrix(wl_df, abstract_concepts):
    # Lọc score == 1 trong wl_df để dễ truy cập
    matched = wl_df[wl_df['score'] == 1]

    # Tạo set {(text, keyword)} đã match
    matched_pairs = set(zip(matched['text'], matched['keyword']))

    # Lấy danh sách unique texts
    texts = wl_df['text'].unique()

    # Kết quả
    results = []

    for text in tqdm(texts, total=len(texts), desc="Abstract concept weak labeling"):
        for concept in abstract_concepts:
            concept_name = concept['abstract_concept_name']
            concept_keywords = concept['keywords']

            # Nếu có ít nhất một keyword trong concept xuất hiện trong matched_pairs → score = 1
            score = int(any((text, kw) in matched_pairs for kw in concept_keywords))

            results.append((text, concept_name, score))

    return pd.DataFrame(results, columns=["text", "abstract_concept", "score"])


In [ ]:
abstract_df = aggregate_full_concept_matrix(kw_wl_df, abstract_concepts)

In [ ]:
abstract_df.sample(5)

In [ ]:
abstract_df.value_counts('score')

In [ ]:
from utils.data_io import join_path, save_csv

In [ ]:
save_csv(kw_wl_df, dir=join_path(dataset, 'weak_label_data'), file_name='keyword_wl.csv')

In [ ]:
save_csv(abstract_df, dir=join_path(dataset, 'weak_label_data'), file_name='abstract_wl.csv')